In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://academiccalendar.dal.ca/Catalog/ViewCatalog.aspx?pageid=viewcatalog&catalogid=111&chapterid=6741&topicgroupid=29561&loaduseredits=False"

In [3]:
'''
uclient = ureq(url)
page_html = uclient.read()
uclient.close()
page_soup = soup(page_html, "html.parser")
'''

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts (for driver to click on)

In [5]:
page_html = driver.page_source
type(page_html)

str

In [6]:
link_texts = re.findall("[A-Z]{4} [0-9]{4}", page_html)
link_texts = list(dict.fromkeys(link_texts))
link_texts

['MECH 3010',
 'MECH 3500',
 'MECH 3805',
 'CPST 3030',
 'ENGM 3356',
 'MATL 3500',
 'MECH 3020',
 'MECH 3305',
 'MECH 3660',
 'MECH 3705',
 'MECH 3900',
 'MECH 4015',
 'MECH 4300',
 'MECH 4600',
 'MECH 4805',
 'MECH 4025',
 'MECH 4500',
 'MECH 4900']

# 2. Test run - scrape the first course

In [7]:
link = driver.find_element_by_link_text(link_texts[0])
link.click()
time.sleep(2)
driver.page_source

'<html xmlns="http://www.w3.org/1999/xhtml" class=" webkit chrome win js js"><head id="ctl00_ctl00_head"><title>\n\tDalhousie Academic Calendars - View Calendar\n</title><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="stylesheet" type="text/css" href="https://use.typekit.net/cyj6tyg.css">\n\t\t<script src="https://connect.facebook.net/signals/config/2125532231026570?v=2.9.43&amp;r=stable" async=""></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script type="text/javascript" async="" src="//www.googleadservices.com/pagead/conversion_async.js"></script><script type="text/javascript" async="" src="//www.googleadservices.com/pagead/conversion_async.js"></script><script type="text/javascript" async="" src="//www.googleadservices.com/pagead/conversion_async.js"></script><script type="text/javascript" async="" src="//www.googleadservices.com/pagead/conversion_async.js">

In [8]:
page_soup = soup(driver.page_source, 'lxml')
container = page_soup.find("div", {"class": "maincontent"})
container.text

'\n\nCo-op Program - Machine Design: Loading of Components\nMECH 3010\xa0\xa0 Machine Design: Loading of Components \n\nCREDIT HOURS: 3\n\nThe application of basic methods and concepts of strength of materials to machine design including design concepts, stress, and theories of failure is developed. Topics include: load analysis, materials, static stresses, strain and deflection, failure, impact, fatigue, surface damage. Applications include: screw fastenings, springs.\n\n\n\n\nFORMAT: LectureLabTutorial\n\n\n\n\n\n\n\n\n\n\n\n\t\t\n\nPREREQUISITES: MECH 2100.03 or ENGI 2203.03, ENGI 2400.03\n\nBack\n'

In [9]:
container.find("b").text.split("\xa0\xa0")

['MECH 3010', ' Machine Design: Loading of Components']

In [10]:
re.split(r'\n+', container.text)

['',
 'Co-op Program - Machine Design: Loading of Components',
 'MECH 3010\xa0\xa0 Machine Design: Loading of Components ',
 'CREDIT HOURS: 3',
 'The application of basic methods and concepts of strength of materials to machine design including design concepts, stress, and theories of failure is developed. Topics include: load analysis, materials, static stresses, strain and deflection, failure, impact, fatigue, surface damage. Applications include: screw fastenings, springs.',
 'FORMAT: LectureLabTutorial',
 '\t\t',
 'PREREQUISITES: MECH 2100.03 or ENGI 2203.03, ENGI 2400.03',
 'Back',
 '']

In [11]:
driver.back()

# 3. Test run complete. Implement Automation script to scrape all courses

In [20]:
course_codes = []
course_names = []
course_descs = []
counter = 0

for link_text in link_texts[::-1]: #when links are clicked from top to bottom some overlapping occurs in the links that lead to errors. So click links from bottom to top
    
    #go to course webpage
    link = driver.find_element_by_link_text(link_text)
    time.sleep(2)
    link.click()
    time.sleep(3)
    page_source = driver.page_source
    
    #handoff to beautiful soup
    page_soup = soup(page_source, 'lxml')
    container = page_soup.find("div", {"class": "maincontent"})
    
    course_code = container.find("b").text.split("\xa0\xa0")[0].strip()
    course_name = container.find("b").text.split("\xa0\xa0")[1].strip()
    
    texts = re.split(r'\n+', container.text)
    texts = [text for text in texts if (len(text)>5) and ("REQUISITES:" not in text) and ("FORMAT" not in text)]
    
    #assume the course description is the longest string in the texts list
    course_desc = max(texts, key = len).strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    counter += 1
    print("Scraped ", course_code)
    
    #go back to course list webpage
    driver.back()
    time.sleep(3)
    
print("Finished scraping {} courses".format(counter))

Scraped  MECH 4900
Scraped  MECH 4500
Scraped  MECH 4025
Scraped  MECH 4805
Scraped  MECH 4600
Scraped  MECH 4300
Scraped  MECH 4015
Scraped  MECH 3900
Scraped  MECH 3705
Scraped  MECH 3660
Scraped  MECH 3305
Scraped  MECH 3020
Scraped  MATL 3500
Scraped  ENGM 3356
Scraped  CPST 3030
Scraped  MECH 3805
Scraped  MECH 3500
Scraped  MECH 3010
Finished scraping 18 courses


In [21]:
len(course_codes)

18

In [22]:
len(course_names)

18

In [23]:
len(course_descs)

18

# 4. Inspect, clean, and write to CSV

In [24]:
course_codes

['MECH 4900',
 'MECH 4500',
 'MECH 4025',
 'MECH 4805',
 'MECH 4600',
 'MECH 4300',
 'MECH 4015',
 'MECH 3900',
 'MECH 3705',
 'MECH 3660',
 'MECH 3305',
 'MECH 3020',
 'MATL 3500',
 'ENGM 3356',
 'CPST 3030',
 'MECH 3805',
 'MECH 3500',
 'MECH 3010']

In [25]:
course_names

['Systems II',
 'Vibrations',
 'Design Project II',
 'Thermo-Fluid Engineering III',
 'Engineering Measurements',
 'Stress Analysis',
 'Design Project I',
 'Systems I',
 'Heat Transfer',
 'Finite Element Method in Mechanical Design',
 'Fluid Mechanics',
 'Machine Design: Power Components',
 'Materials Engineering',
 'Numerical Methods and Partial Differential Equations',
 'Engineering in Society II',
 'Mechanical Engineering Thermodynamics',
 'Dynamics of Machines',
 'Machine Design: Loading of Components']

In [26]:
course_descs

['Response characteristics of open loop and feedback control systems are studied. Various controller types and their uses are analyzed. Techniques such as root-locus diagrams and Bode & Nichols plots are used for stability and performance evaluation. Digital simulations and experiments on computer-based control systems are done in the laboratory portion.',
 'Single and multiple degree of freedom lumped parameter systems subjected to harmonic and transient excitation are examined. Analytical as well as numerical solutions are covered. Vibrations of continuous systems such as beams and shafts are introduced. Laboratory experiments deal with vibration of lumped parameter physical models as well as vibrations of rotating machinery. Vibration control in industrial applications is emphasized and the effects of whole body vibration on humans is treated as a safety issue.',
 'This course is a direct continuation of Design Project I leading to the implementation of the student team design. All 

In [27]:
import pandas as pd

course_codes = course_codes[::-1]
course_names = course_names[::-1]
course_descs = course_descs[::-1]

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df.to_csv('Dalhousie_MechEng_Core_(Year3-4)_Courses.csv', index = False)

In [28]:
driver.quit()